In [6]:
import matplotlib.pyplot as plt
import numpy as np
import qiskit as qs


from numpy.random import seed
seed(1)
from datetime import datetime
from scipy.optimize import minimize
from logger import Logger
# from tools.optimizationtools import SPSA_optimization, SPSA_calibration

SHOTS = 1024 

Q_program = qs.QuantumProgram()


def build_discriminator_gates(phi, circuit, q_register, c_register):
    # phase - cnot - phase - cnot - measurement
    circuit.rx(phi[0], q_register[0])
    circuit.rx(phi[1], q_register[1])
    circuit.cx(q_register[0], q_register[1])
    circuit.rz(phi[2], q_register[0])
    circuit.rz(phi[3], q_register[1])
    circuit.cx(q_register[0], q_register[1])
    circuit.measure(q_register[0], c_register[0])


def build_generator_gates(theta, circuit, q_register):
    circuit.x(q_register[0])
    circuit.u1(theta[0], q_register[0])


def build_discriminator_generator(theta, phi):
    d_g_quantum_register = Q_program.create_quantum_register('d_g_qr', 2)
    d_g_classical_register = Q_program.create_classical_register('d_g_cr', 1)
    d_g_circuit = Q_program.create_circuit('d_g_circuit', [d_g_quantum_register], [d_g_classical_register])

    build_generator_gates(theta, d_g_circuit, d_g_quantum_register)
    build_discriminator_gates(phi, d_g_circuit, d_g_quantum_register, d_g_classical_register)


def build_discriminator(phi):
    d_quantum_register = Q_program.create_quantum_register('d_qr', 2)
    d_classical_register = Q_program.create_classical_register('d_cr', 1)
    d_circuit = Q_program.create_circuit('d_circuit', [d_quantum_register], [d_classical_register])

    # prepare target state
    d_circuit.x(d_quantum_register[0])
    d_circuit.z(d_quantum_register[0])

    build_discriminator_gates(phi, d_circuit, d_quantum_register, d_classical_register)


def discriminator_probability(theta, phi, circuit_name, shots):
    if circuit_name == 'd_circuit':
        build_discriminator(phi)
    else:
        build_discriminator_generator(theta, phi)
    stats_generator = Q_program.execute([circuit_name], backend='local_qasm_simulator', shots=shots)
    stats = stats_generator.get_counts(circuit_name)
    return stats['1'] / shots if '1' in stats else 0.0

# def value_function(theta, phi):
#     d_prob = discriminator_probability(theta, phi, circuit_name='d_circuit', shots=shots)
#     d_g_prob = discriminator_probability(theta, phi, circuit_name='d_g_circuit', shots=shots)
#     # vf = np.log(d_prob) + np.log(1 - d_g_prob)
#     vf = d_prob - d_g_prob
#     return vf


def loss_discriminator(phi, theta):
    d_prob = discriminator_probability(theta, phi, circuit_name='d_circuit', shots=SHOTS)
    d_g_prob = discriminator_probability(theta, phi, circuit_name='d_g_circuit', shots=SHOTS)
    vf = 2*d_prob - d_g_prob
    return - vf


def loss_generator(theta, phi):
    d_g_prob = discriminator_probability(theta, phi, circuit_name='d_g_circuit', shots=SHOTS)
    return - d_g_prob


# options = {'maxiter': 5}
options = None

theta_max = 2 * np.pi
phi_max = 2 * np.pi

theta = np.array([np.random.uniform(0, theta_max)])
theta_bounds = [(0.0, theta_max)]

phi = np.array([np.random.uniform(0.0, phi_max) for _ in range(4)])
phi_bounds = [(0.0, phi_max) for _ in range(4)]

# method = 'L-BFGS-B'
# method = 'TNC'
method ='SLSQP'

max_iter = 1000

print(max_iter)


# make a plot of the loss landscape

# x = [i * 2 * np.pi / 100 for i in range(100)]
# y = [loss_generator([theta], phi) for theta in x]
# plt.scatter(x, y)
# plt.savefig('generator_loss.pdf')

now = datetime.now().replace(microsecond=0).isoformat()


logger = Logger('logs/{}'.format(now))

# scipy

# for t in range(max_iter):
#     logger.log_scalar('theta', theta[0], t)
#
#     res_phi = minimize(loss_discriminator, args=(theta), x0=phi, method=method, options=options, bounds=phi_bounds)
#     phi = res_phi['x']
#     res_theta = minimize(loss_generator, args=(phi), x0=theta, method=method, options=options, bounds=theta_bounds)
#     theta = res_theta['x']

# SPSA

initial_c = 0.01
target_update = 2 * np.pi * 0.1
max_trials = 10

gen_params = SPSA_calibration(generator_obj, theta, initial_c, target_update, stat=25)
dis_params = SPSA_calibration(discriminator_obj, phi, initial_c, target_update, stat=25)


for t in range(max_iter):

    logger.log_scalar('theta', theta[0], t)

    def generator_obj(_theta):
        return loss_generator(_theta, phi)

    theta = SPSA_optimization(generator_obj, initial_theta=theta, SPSA_parameters=gen_params, max_trials=max_trials)[1]

    def discriminator_obj(_phi):
        return loss_discriminator(_phi, theta)

    phi = SPSA_optimization(discriminator_obj, initial_theta=phi, SPSA_parameters=dis_params, max_trials=max_trials)[1]




AttributeError: module 'qiskit' has no attribute 'QuantumProgram'